In [ ]:
import pandas as pd
from datetime import datetime
import base64

# Composition Input files Formatting

In [ ]:
rate = 'pr' # rr or pr
file_path = '/Users/steve/Documents/GitHub/Rebalancing/DOGE/DOGEUSD_RR.csv' # File path to Theo's file
export_path = '/Users/steve/Documents/GitHub/Rebalancing/DOGE/' # Path for output 

In [ ]:
df = pd.read_csv(file_path)
df[['base', 'quote']] = df['pair'].str.split('-', expand=True)
date_columns = ['start_data_period', 'end_data_period', 'start_effective_period', 'end_effective_period']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%m/%d/%y').dt.strftime('%Y-%m-%d')
df['exchanges_post_vol_filter'] = df['exchanges_post_vol_filter'].apply(lambda x: ','.join(eval(x)))
df['percentiles'] = 100

if rate == 'rr': 
    configs = [
        {'name': 'rr_rt_composition.csv', 'publication_freq': 'PT5S', 'calc_window': '15,20,30,60,120,300', 'cron': '*/5 * * * * * *'},
        {'name': 'rr_fx_composition_sgp.csv', 'publication_freq': 'P1D', 'calc_window': '300,600,900,1200,1800,3600', 'cron': '0 8 * * *'},
        {'name': 'rr_fx_composition_ldn.csv', 'publication_freq': 'P1D', 'calc_window': '300,600,900,1200,1800,3600', 'cron': '0 16 * * *'},
        {'name': 'rr_fx_composition_nyc.csv', 'publication_freq': 'P1D', 'calc_window': '300,600,900,1200,1800,3600', 'cron': '0 20 * * *'}
    ]
elif rate == 'pr': 
    configs = [
        {'name': 'pr_rt_composition.csv', 'publication_freq': 'PT5S', 'calc_window': '300', 'cron': '*/5 * * * * * *'},
        {'name': 'pr_fx_composition_sgp.csv', 'publication_freq': 'P1D', 'calc_window': '3600', 'cron': '0 8 * * *'},
        {'name': 'pr_fx_composition_ldn.csv', 'publication_freq': 'P1D', 'calc_window': '3600', 'cron': '0 16 * * *'},
        {'name': 'pr_fx_composition_nyc.csv', 'publication_freq': 'P1D', 'calc_window': '3600', 'cron': '0 20 * * *'}
    ]


for config in configs:
    df_temp = df.copy()
    df_temp['publication_freq'] = config['publication_freq']
    df_temp['calc_window'] = config['calc_window']
    df_temp['cron'] = config['cron']
    df_temp = df_temp[['base', 'quote', 'start_data_period', 'end_data_period', 'exchanges_post_vol_filter', 'percentiles', 'publication_freq', 'calc_window', 'cron']]

    # Define file path
    output_file_path = export_path + f'{config["name"]}'

    # Write data to CSV in the second format
    df_temp.to_csv(output_file_path, sep=';', index=False)

    # Read the file and encode in Base64
    with open(output_file_path, 'rb') as file:
        base64_encoding = base64.b64encode(file.read()).decode('utf-8')

    # Print the file name and Base64 encoding
    print(f'{config["name"]}: {base64_encoding}')

# Buffering Formatting: 

In [ ]:
if rate == 'rr': 
    configs = [
        {'name': 'rr_rt_buffer.csv', 'cron': '*/5 * * * * * *'},
        {'name': 'rr_fx_buffer_sgp.csv', 'cron': '0 8 * * *'},
        {'name': 'rr_fx_buffer_ldn.csv', 'cron': '0 16 * * *'},
        {'name': 'rr_fx_buffer_nyc.csv', 'cron': '0 20 * * *'}
    ]

    for config in configs:
        df_temp = df.copy()
        df_temp['cron'] = config['cron']
        df_temp = df_temp[['base', 'quote','cron']]

        # Define file path
        output_file_path = export_path + f'{config["name"]}'

        df_temp = df_temp.iloc[0:1]

        # Write data to CSV in the second format
        df_temp.to_csv(output_file_path, sep=';', index=False)

        # Read the file and encode in Base64
        with open(output_file_path, 'rb') as file:
            base64_encoding = base64.b64encode(file.read()).decode('utf-8')

        # Print the file name and Base64 encoding
        print(f'{config["name"]}: {base64_encoding}')